In [297]:
import pandas as pd

In [298]:
data = pd.read_csv("/Data/Spotify_Youtube.csv")

## Data Exploration

In [299]:
data.shape

(20723, 27)

In [300]:
data.columns

Index(['Artist', 'Url_spotify', 'Track', 'Album', 'Album_type', 'Uri',
       'Danceability', 'Energy', 'Key', 'Loudness', 'Speechiness',
       'Acousticness', 'Instrumentalness', 'Liveness', 'Valence', 'Tempo',
       'Duration_ms', 'Url_youtube', 'Title', 'Channel', 'Views', 'Likes',
       'Comments', 'Description', 'Licensed', 'official_video', 'Stream'],
      dtype='object')

In [301]:
data.dtypes

Artist               object
Url_spotify          object
Track                object
Album                object
Album_type           object
Uri                  object
Danceability        float64
Energy              float64
Key                 float64
Loudness            float64
Speechiness         float64
Acousticness        float64
Instrumentalness    float64
Liveness            float64
Valence             float64
Tempo               float64
Duration_ms         float64
Url_youtube          object
Title                object
Channel              object
Views               float64
Likes               float64
Comments            float64
Description          object
Licensed             object
official_video       object
Stream              float64
dtype: object

## DATA PREPROCESSING

Eliminate unnecessary columns from the dataset for our analysis by removing Url_spotify, Uri, Key, Url_youtube, and Description.

In [280]:
data = data.drop(columns = ['Url_spotify','Uri','Key','Url_youtube','Description'])

Examine the dataset for the presence of null values and calculate the total count of null values for each column, providing insights into the data's completeness and potential data quality issues.

In [302]:
data.isnull().sum()

Artist                0
Url_spotify           0
Track                 0
Album                 0
Album_type            0
Uri                   0
Danceability          2
Energy                2
Key                   2
Loudness              2
Speechiness           2
Acousticness          2
Instrumentalness      2
Liveness              2
Valence               2
Tempo                 2
Duration_ms           2
Url_youtube         470
Title               470
Channel             470
Views               470
Likes               541
Comments            569
Description         876
Licensed            470
official_video      470
Stream              577
dtype: int64

Manage null values in data for improved data quality and analysis 

In [303]:
for i in data.columns:
  if data[i].isnull().sum()>0:
    if data[i].dtypes == "float64":
      data[i].fillna(0, inplace = True)
    else :
      data[i].fillna("NA", inplace = True)  

In [304]:
data.isnull().sum()

Artist              0
Url_spotify         0
Track               0
Album               0
Album_type          0
Uri                 0
Danceability        0
Energy              0
Key                 0
Loudness            0
Speechiness         0
Acousticness        0
Instrumentalness    0
Liveness            0
Valence             0
Tempo               0
Duration_ms         0
Url_youtube         0
Title               0
Channel             0
Views               0
Likes               0
Comments            0
Description         0
Licensed            0
official_video      0
Stream              0
dtype: int64

Identify and eliminate duplicate records in the dataset while retaining the first occurrence of each unique value. This ensures data integrity by removing redundant information and maintaining the original data structure.

In [305]:
data.duplicated().sum()

5

In [306]:
data.drop_duplicates(keep='first',inplace=True)

In [307]:
data.shape

(20718, 27)

**Convert the duration in milliseconds to minutes, facilitating a clearer comprehension and representation of time intervals in a more user-friendly format.**


In [308]:
data['Duration_ms'].head()

0    222640.0
1    200173.0
2    215150.0
3    233867.0
4    340920.0
Name: Duration_ms, dtype: float64

In [309]:
data['Duration_ms'] = data['Duration_ms']/60000

In [310]:
data['Duration_ms'].head()

0    3.710667
1    3.336217
2    3.585833
3    3.897783
4    5.682000
Name: Duration_ms, dtype: float64

**Change the name of the modified column to "Duration_min" to accurately reflect the conversion from milliseconds to minutes, providing a more descriptive and meaningful representation of the data.**

In [311]:
data.rename(columns = {"Duration_ms":"Duration_min"}, inplace=True)

**Eliminate track names that are deemed irrelevant and begin with the "?" character, ensuring the dataset only includes relevant and meaningful track information for further analysis or processing.**

In [312]:
data["Track"].str.startswith("?").sum()

124

In [313]:
remove_rows = []
for row in range(data.shape[0]):
  try:
    if data.loc[row]["Track"].startswith("?"):
      remove_rows.append(row)
  except:
    continue

In [314]:
data.drop(index = remove_rows, inplace=True)

In [315]:
data.shape

(20594, 27)

In [316]:
data["Track"].str.startswith("?").sum()

0

**Compute the Energy to Liveness ratio for each track, quantifying the relationship between energy and liveliness attributes. The resulting ratios are then stored in a column named 'EnergyLiveness' for further analysis or interpretation.**

In [317]:
data.columns

Index(['Artist', 'Url_spotify', 'Track', 'Album', 'Album_type', 'Uri',
       'Danceability', 'Energy', 'Key', 'Loudness', 'Speechiness',
       'Acousticness', 'Instrumentalness', 'Liveness', 'Valence', 'Tempo',
       'Duration_min', 'Url_youtube', 'Title', 'Channel', 'Views', 'Likes',
       'Comments', 'Description', 'Licensed', 'official_video', 'Stream'],
      dtype='object')

In [318]:
data["EnergyLiveness"] = data['Energy']/data['Liveness']

In [319]:
data.columns[-1]

'EnergyLiveness'

**Modify the data type of the 'views' column to float, enabling numerical operations and facilitating its utilization in subsequent analysis or calculations requiring floating-point values.**

In [320]:
data["Views"].dtypes

dtype('float64')

In [321]:
data = data.astype({"Views":"float"})

In [322]:
data.Views.dtypes

dtype('float64')

**Analyze the 'views' and 'stream' columns to determine the dominant platform (YouTube or Spotify) on which a song track was most played. Create a new column called 'most_playedon' with values 'Spotify' or 'YouTube' indicating the platform with the highest play count for each song track.**

In [323]:
for row in range(20723):
  try:
    if data.loc[row]["Stream"]> data.loc[row]["Views"]:
      data.loc[row, "most_playedon"] = "Spotify"
    else:
      data.loc[row,"most_playedon"] = "Youtube"
    #using loc instaed of iloc to avoid copy warning, as this will make changes in the original dataframe
  except KeyError:
    continue    

In [324]:
data.most_playedon.value_counts()

Spotify    15694
Youtube     4900
Name: most_playedon, dtype: int64

**Export the data to a CSV file named "cleaned_dataset.csv" and enable downloading by providing a clickable file name, allowing users to access and retrieve the file with ease.**

In [325]:
data.to_csv("cleaned_dataset.csv", encoding ="utf-8", index=False)

###SQL

**Which is the most viewed song track on youtube?**

In [1]:
## SQL
SELECT C.Track, C.Views
FROM cleaned_dataset C
ORDER BY C.Views DESC
LIMIT 1; 

**Which Song track is streamed most on Spotify?**

In [ ]:
## SQL
SELECT C.Track, C.Stream
FROM cleaned_dataset C
ORDER BY C.Stream DESC
LIMIT 1;

**EnergyLiveness ratio is one of the popular ways to measure the quality of the song, which are the top 5 songs that have the highest energyliveness ratio.**

In [ ]:
## SQL
SELECT C.Track, C.EnergyLiveness
FROM cleaned_dataset C
ORDER BY C.EnergyLiveness DESC
LIMIT 5;

**let us assume a situation where an artist named Black Eyed Peas wants to analyze his songs. The artist wants to know which platform is capable of keeping his song track more engaged. To check this he assigns you this task and wants you to report to him where his song tracks are more played on. compare the platforms.**

In [ ]:
## SQL
SELECT COUNT(C.most_playedon) most_played_count, C.most_playedon
FROM cleaned_dataset C
WHERE C.Artist = "Black Eyed Peas"
GROUP BY C.most_playedon

**Gorillaz wants to know their most liked song on youtube. Report to them with their most liked song along with the Energy and Tempo of the song.**

In [ ]:
## SQL
SELECT C.Track, C.Likes, C.Energy, C.Tempo
FROM cleaned_dataset C
WHERE C.Artist ="Gorillaz"
ORDER BY C.Likes DESC
LIMIT 1

**Which Album types are more prominent on Spotify and Youtube**

In [ ]:
## SQL
SELECT C.Album_type, COUNT(C.Album_type) type_count
FROM cleaned_dataset C
GROUP BY C.Album_type
ORDER BY type_count DESC

**Spotify's most loved song tracks are to be declared soon. Help Spotify choose the top 5 most streamed+youtube viewed song track.**

In [ ]:
## SQL
SELECT DISTINCT(C.Track), (C.Views+C.Stream) AS total
FROM cleaned_dataset C
ORDER BY total
LIMIT 5